## Use FM tooling via Bedrock Converse API to achieve Intelligent Document Proccessing

Document processing is a common challenge faced by many organizations across various industries. It often involves handling large volumes of diverse documents, extracting relevant information, and making informed decisions based on that data. In this notebook, we'll address this real-world problem with a focus on the mortgage application process as an example. We will utilize Anthroipic's multi-modal models like Claude 3 Haiku and Claude 3 Sonnet via the Bedrock Cpnverse API. We will also demonstarte the use of tooling to create an intelligent agent loop. 


An agent loop enables Foundational Models (FM) to tackle complex, multi-stage problems. This design pattern involves a series of iterative steps where the LLM engages in multiple interactions, executes various functions, and analyzes their outcomes. This process continues until the desired end result is accomplished, allowing the AI to navigate through intricate problem-solving scenarios.

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import boto3, json
from constants import ModelIDs, Temperature, ToolConfig
from utils import FileUtility
from bedrock_util import BedrockUtils
from tools import IDPTools

message_list = []

sonnet_model_id = ModelIDs.anthropic_claude_3_sonnet
haiku_model_id = ModelIDs.anthropic_claude_3_haiku

temp_focused = Temperature.FOCUSED
temp_balanced = Temperature.BALANCED

sonnet_bedrock_utils = BedrockUtils(model_id=sonnet_model_id)
haiku_bedrock_utils = BedrockUtils(model_id=haiku_model_id)

tool = IDPTools()

In [3]:
def limited_print(data, limit=1000):
    print(str(data)[:limit] + "..." if len(str(data)) > limit else data)

In [4]:
source_folder = "forms"
target_folder = "images/_extracted"
source_bucket = "bedrock-tool-use-789068066945"
# source_key = "forms/urls_filled.pdf"
# source_key = "forms/urla_1.png"
# source_key = "new-jersey-drivers-license.png"
# source_key = "loan-applications/loan-package.zip"
source_key = "loan-applications/combined_application.pdf"

In [8]:
messages = haiku_bedrock_utils.run_loop(
            f"Start document processing of loan application file {source_key} in s3 bucket {source_bucket}", 
            ToolConfig.COT,
            tool.get_tool_result
        )

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=4))

Invoking Bedrock model anthropic.claude-3-haiku-20240307-v1:0...
Input Tokens: 2269
Output Tokens: 136
Using tool download_application_package
using temp folder temp_erzuj
Downloaded file is not a zip file: temp_erzuj/combined_application.pdf
Invoking Bedrock model anthropic.claude-3-haiku-20240307-v1:0...
Input Tokens: 2432
Output Tokens: 125
Using tool pdf_to_images
using temp folder temp_erzuj
Invoking Bedrock model anthropic.claude-3-haiku-20240307-v1:0...
Input Tokens: 2638
Output Tokens: 162
Using tool classify_documents
using temp folder temp_erzuj
Invoking Bedrock model anthropic.claude-3-5-sonnet-20240620-v1:0...
Input Tokens: 12471
Output Tokens: 190
Invoking Bedrock model anthropic.claude-3-haiku-20240307-v1:0...
Input Tokens: 3073
Output Tokens: 252
Using tool check_required_documents
using temp folder temp_erzuj
doc list is URLA, DRIVERS_LICENSE, UNK
All required documents are present.
Invoking Bedrock model anthropic.claude-3-haiku-20240307-v1:0...
Input Tokens: 3340
Outp

In [57]:
message_list = [
    {
        "role": "user",
        "content": [
            {
                "text": "Start document processing of loan application file loan-applications/combined_application.pdf in s3 bucket bedrock-tool-use-789068066945"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "Okay, let's process the loan application file. Here are the steps:"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_aAEqh9d-R_ygU3G2TmaFyA",
                    "name": "download_application_package",
                    "input": {
                        "source_bucket": "bedrock-tool-use-789068066945",
                        "source_key": "loan-applications/combined_application.pdf",
                        "target_folder": "./"
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_aAEqh9d-R_ygU3G2TmaFyA",
                    "content": [
                        {
                            "json": {
                                "result": [
                                    "temp/combined_application.pdf"
                                ]
                            }
                        }
                    ]
                }
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "The download_application_package function downloaded the PDF file from the S3 bucket and saved it locally as 'temp/combined_application.pdf'.\n\nNext, let's convert the PDF to images:"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_CdavlhbkRv-UTLaGPKhFaA",
                    "name": "pdf_to_images",
                    "input": {
                        "pdf_path": "temp/combined_application.pdf"
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_CdavlhbkRv-UTLaGPKhFaA",
                    "content": [
                        {
                            "json": {
                                "result": [
                                    "temp/fa46c7ad-2cce-4305-bf9b-258e97e26cd6.png",
                                    "temp/8302185b-cac4-4345-a596-68398d7a1a3b.png",
                                    "temp/189ce272-45c2-4894-ad15-17d2a0654c9c.png",
                                    "temp/7b471d1d-8a37-428c-914f-7f07a301e3d1.png",
                                    "temp/e2167b62-2a41-4b30-a198-e3449c5d68ca.png",
                                    "temp/e9fa9a66-faa1-4eb5-bf91-ca04ebf0dcfa.png",
                                    "temp/35bf473e-8664-4417-9040-789e8350a1e1.png",
                                    "temp/5e6a26ee-7bfc-4bda-a6ea-30641d312337.png",
                                    "temp/5d829e87-518e-411c-b681-a892e0a232ba.png",
                                    "temp/9244b13f-e070-431f-919c-3a572a062675.png",
                                    "temp/026f7a50-64a3-455b-9d79-0f72631e1ef6.png"
                                ]
                            }
                        }
                    ]
                }
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "The pdf_to_images function converted the PDF file to a series of image files, which are stored in the 'temp' folder.\n\nNow let's classify the documents in the loan application package:"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_1_wt426ST5e6Q05CmYWcCQ",
                    "name": "classify_documents",
                    "input": {
                        "document_paths": [
                            "temp/fa46c7ad-2cce-4305-bf9b-258e97e26cd6.png",
                            "temp/8302185b-cac4-4345-a596-68398d7a1a3b.png",
                            "temp/189ce272-45c2-4894-ad15-17d2a0654c9c.png",
                            "temp/7b471d1d-8a37-428c-914f-7f07a301e3d1.png",
                            "temp/e2167b62-2a41-4b30-a198-e3449c5d68ca.png",
                            "temp/e9fa9a66-faa1-4eb5-bf91-ca04ebf0dcfa.png",
                            "temp/35bf473e-8664-4417-9040-789e8350a1e1.png",
                            "temp/5e6a26ee-7bfc-4bda-a6ea-30641d312337.png",
                            "temp/5d829e87-518e-411c-b681-a892e0a232ba.png",
                            "temp/9244b13f-e070-431f-919c-3a572a062675.png",
                            "temp/026f7a50-64a3-455b-9d79-0f72631e1ef6.png"
                        ]
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_1_wt426ST5e6Q05CmYWcCQ",
                    "content": [
                        {
                            "json": {
                                "result": [
                                    [
                                        {
                                            "role": "assistant",
                                            "content": [
                                                {
                                                    "text": "{'docs': [{'type': ['URLA'], 'start_page_number': [1], 'end_page_number': [6]}, {'type': ['DRIVERS_LICENSE'], 'start_page_number': [7], 'end_page_number': [7]}, {'type': ['W2'], 'start_page_number': [8], 'end_page_number': [8]}]}"
                                                }
                                            ]
                                        }
                                    ]
                                ]
                            }
                        }
                    ]
                }
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "The classify_documents function has identified the following document types in the loan application package:\n- URLA (Uniform Residential Loan Application) on pages 1-6\n- Driver's License on page 7\n- W2 on page 8\n\nNext, let's check if all the required documents are present:"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_v4N5MsgHSAaCoQGqhHRPag",
                    "name": "check_required_documents",
                    "input": {
                        "classified_documents": "{'URLA': ['temp/fa46c7ad-2cce-4305-bf9b-258e97e26cd6.png', 'temp/8302185b-cac4-4345-a596-68398d7a1a3b.png', 'temp/189ce272-45c2-4894-ad15-17d2a0654c9c.png', 'temp/7b471d1d-8a37-428c-914f-7f07a301e3d1.png', 'temp/e2167b62-2a41-4b30-a198-e3449c5d68ca.png', 'temp/e9fa9a66-faa1-4eb5-bf91-ca04ebf0dcfa.png'], 'DRIVERS_LICENSE': ['temp/35bf473e-8664-4417-9040-789e8350a1e1.png'], 'W2': ['temp/5e6a26ee-7bfc-4bda-a6ea-30641d312337.png']}"
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_v4N5MsgHSAaCoQGqhHRPag",
                    "content": [
                        {
                            "json": {
                                "result": [
                                    "Error: Invalid JSON string provided"
                                ]
                            }
                        }
                    ]
                }
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "Oops, it looks like the check_required_documents function expects a valid JSON object, not a string. Let me try that again:"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_Ful_qX72SxiN3l-lzC-GRg",
                    "name": "check_required_documents",
                    "input": {
                        "classified_documents": {
                            "URLA": [
                                "temp/fa46c7ad-2cce-4305-bf9b-258e97e26cd6.png",
                                "temp/8302185b-cac4-4345-a596-68398d7a1a3b.png",
                                "temp/189ce272-45c2-4894-ad15-17d2a0654c9c.png",
                                "temp/7b471d1d-8a37-428c-914f-7f07a301e3d1.png",
                                "temp/e2167b62-2a41-4b30-a198-e3449c5d68ca.png",
                                "temp/e9fa9a66-faa1-4eb5-bf91-ca04ebf0dcfa.png"
                            ],
                            "DRIVERS_LICENSE": [
                                "temp/35bf473e-8664-4417-9040-789e8350a1e1.png"
                            ],
                            "W2": [
                                "temp/5e6a26ee-7bfc-4bda-a6ea-30641d312337.png"
                            ]
                        }
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_Ful_qX72SxiN3l-lzC-GRg",
                    "content": [
                        {
                            "json": {
                                "result": [
                                    {
                                        "role": "assistant",
                                        "content": [
                                            {
                                                "text": "[\"DRIVERS_LICENSE\"]"
                                            }
                                        ]
                                    }
                                ]
                            }
                        }
                    ]
                }
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "The check_required_documents function indicates that the Drivers License document is the only required document that is missing from the loan application package.\n\nSince the Drivers License document is missing, we should reject the incomplete application:"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_bu_pecSmQGqVyqmkTeYyDg",
                    "name": "reject_incomplete_application",
                    "input": {
                        "missing_documents": [
                            "DRIVERS_LICENSE"
                        ]
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_bu_pecSmQGqVyqmkTeYyDg",
                    "content": [
                        {
                            "json": {
                                "result": [
                                    [
                                        {
                                            "role": "assistant",
                                            "content": [
                                                {
                                                    "text": "Dear Valued Customer,\n\nI hope this message finds you well. I am writing to follow up on your mortgage application. Upon reviewing your submitted documents, I noticed that we are missing a copy of your DRIVERS_LICENSE.\n\nTo proceed with your application, I kindly request that you provide a copy of your valid driver's license as soon as possible. This document is an essential part of the application process and will help us verify your identity and eligibility.\n\nPlease feel free to send the requested document via email, fax, or by uploading it to our secure online portal. If you have any questions or need assistance, please don't hesitate to reach out to me.\n\nThank you for your prompt attention to this matter. I look forward to receiving the missing documentation and continuing to assist you with your mortgage needs.\n\nBest regards,\n[Your Name]\nMortgage Agent"
                                                }
                                            ]
                                        }
                                    ]
                                ]
                            }
                        }
                    ]
                }
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "The reject_incomplete_application function has generated a rejection letter informing the customer that their application is incomplete due to the missing Drivers License document. The customer is requested to provide the missing document as soon as possible to continue the application process."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "text": "what are the file paths for the URLA document? There should be 9 files paths. get the information from input ot the check_required_documents tool"
            }
        ]
    }
]

system_message = [
            {
                "text": (
                    "Do not make up information. "
                    "Before generating the information check multiple times if the information is correct. "
                    "If needed go back and read the information provided to inderstand what is being asked." 
                    "look carefully thorugh every conversation and double check your answer."
                )
            }
        ]

[autoreload of tools failed: Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/sagemaker-user/tools.py", line 45
    pting(f"using temp folder {TEMP_FOLDER}"
         ^
SyntaxError: '(' was never closed
]


In [78]:
print(message_list)

[{'role': 'user', 'content': [{'text': 'Start document processing of loan application file loan-applications/combined_application.pdf in s3 bucket bedrock-tool-use-789068066945'}]}, {'role': 'assistant', 'content': [{'text': "Okay, let's process the loan application file. Here are the steps:"}, {'toolUse': {'toolUseId': 'tooluse_aAEqh9d-R_ygU3G2TmaFyA', 'name': 'download_application_package', 'input': {'source_bucket': 'bedrock-tool-use-789068066945', 'source_key': 'loan-applications/combined_application.pdf', 'target_folder': './'}}}]}, {'role': 'user', 'content': [{'toolResult': {'toolUseId': 'tooluse_aAEqh9d-R_ygU3G2TmaFyA', 'content': [{'json': {'result': ['temp/combined_application.pdf']}}]}}]}, {'role': 'assistant', 'content': [{'text': "The download_application_package function downloaded the PDF file from the S3 bucket and saved it locally as 'temp/combined_application.pdf'.\n\nNext, let's convert the PDF to images:"}, {'toolUse': {'toolUseId': 'tooluse_CdavlhbkRv-UTLaGPKhFaA',

In [79]:

response = haiku_bedrock_utils.invoke_bedrock(message_list=message_list,
                                              tool_list=ToolConfig.COT,
                                              system_message=system_message)

response_message = response['output']['message']
print(response_message)

Invoking Bedrock model anthropic.claude-3-haiku-20240307-v1:0...
Input Tokens: 4380
Output Tokens: 340
{'role': 'assistant', 'content': [{'text': 'Okay, let me retrieve the file paths for the URLA document from the input to the check_required_documents tool:'}, {'toolUse': {'toolUseId': 'tooluse_t3i87xkIQAef9IDmHPSA-w', 'name': 'check_required_documents', 'input': {'classified_documents': {'URLA': ['temp/fa46c7ad-2cce-4305-bf9b-258e97e26cd6.png', 'temp/8302185b-cac4-4345-a596-68398d7a1a3b.png', 'temp/189ce272-45c2-4894-ad15-17d2a0654c9c.png', 'temp/7b471d1d-8a37-428c-914f-7f07a301e3d1.png', 'temp/e2167b62-2a41-4b30-a198-e3449c5d68ca.png', 'temp/e9fa9a66-faa1-4eb5-bf91-ca04ebf0dcfa.png'], 'DRIVERS_LICENSE': ['temp/35bf473e-8664-4417-9040-789e8350a1e1.png'], 'W2': ['temp/5e6a26ee-7bfc-4bda-a6ea-30641d312337.png']}}}}]}


In [23]:
UNKNOWN_TYPE = "UNK"
DOCUMENT_TYPES = ["URLA", "W2", "PAY_STUB", "DRIVERS_LICENSE", UNKNOWN_TYPE]
TEMP_FOLDER = "temp"
print((
                            "<task>"
                                "You are a document processing agent and you have perfect vision. "
                                "Look through every image presented and categorize them based on the <doc_type_choices>. "
                                "The final output is a json array with an element for each image. "
                                "Each element in the output array contains the classified_doc_type and the file_path"  
                            "</task> "
                            f"<doc_type_choices>{', '.join(DOCUMENT_TYPES)}</doc_type_choices> "
                            "<output_format>"
                                "{'docs': [{'type':value_for_doc_type, 'file_paths': array__for_file_path}]}"
                            "</output_format>"
                            "In this case PS means pay stub, DL means driver's license, and UNK means unknown. "
                            "<important>"
                                "the output has to only be a json array in the <output_format> only. do not include anything else. "
                            "</important>"
                            "<example_output>"
                                "{"
                                  "'documents': ["
                                    "{"
                                      "'type': 'URLA',"
                                      "'files': ["
                                        "'example/1.png',"
                                        "'example/2.png',"
                                        "'example/3.png',"
                                      "]"
                                    "},"
                                    "{"
                                      "'type': 'DL',"
                                      "'files': ["
                                        "'example/4.png'"
                                      "]"
                                    "},"
                                    "{"
                                      "'type': 'W2"
                                      "'files': ["
                                        "'example/5.png'"
                                      "]"
                                    "}"
                                  "]"
                                "}"
                            "</example_output>"
                    ))

<task>You are a document processing agent and you have perfect vision. Look through every image presented and categorize them based on the <doc_type_choices>. The final output is a json array with an element for each image. Each element in the output array contains the classified_doc_type and the file_path</task> <doc_type_choices>URLA, W2, PAY_STUB, DRIVERS_LICENSE, UNK</doc_type_choices> <output_format>{'docs': [{'type':value_for_doc_type, 'file_paths': array__for_file_path}]}</output_format>In this case PS means pay stub, DL means driver's license, and UNK means unknown. <important>the output has to only be a json array in the <output_format> only. do not include anything else. </important><example_output>{'documents': [{'type': 'URLA','files': ['example/1.png','example/2.png','example/3.png',]},{'type': 'DL','files': ['example/4.png']},{'type': 'W2'files': ['example/5.png']}]}</example_output>
